# Reactive TRiii

In [2]:
from radd import build, vis
from radd.toolbox import analyze
from __future__ import division

In [3]:
x1 = {'a_bsl':   0.529353,
'a_pnl': 0.553150,
'xb': 0.817502,
'ssv': -1.104875,
'tr': 0.185667,
'v': 1.208608,
'z': 0.0436727}

x2 = {'a': 0.3964024,
 'ssv': -0.7881214,
 'tr': 0.2915253,
 'v_bsl': 0.93931011,
 'v_pnl': 0.88324705,
 'xb': 1.667787,
 'z': 0.1177627}

x3={'a': 0.44534253,
 'ssv':  -.9073797,
 'tr_bsl':   0.29969275,
 'tr_pnl':   0.31056620,
 'v': 0.965766,
 'xb': 1.52569355,
 'z': 0.1591336}

x4={'a': 0.44534253,
 'ssv':  -1.564015,
 'tr_bsl': 0.19521031,
 'tr_pnl': 0.19703793,
 'v_bsl': 1.80694264,
 'v_pnl': 1.75763984,
 'xb': 1.206618,
 'z': 0.001013259}

x4['tr'] = np.mean([x4['tr_bsl'], x4['tr_pnl']])
x4['v'] = np.mean([x4['v_bsl'], x4['v_pnl']])
x2['v'] = np.mean([x2['v_bsl'], x2['v_pnl']])
x3['tr'] = np.mean([x3['tr_bsl'], x3['tr_pnl']])
x1['a'] = np.mean([x1['a_bsl'], x1['a_pnl']])

In [4]:
finfo_list, yhat_list = [], [],

In [5]:
for i in range(20):
      
      xinits_list = [deepcopy(xdct) for xdct in [x1,x2,x3,x4]]
      parameter = 'tr'
      depends_on = {parameter:'Cond'}
      d = '/'.join(['FinalRe', parameter+'iii'])
      if not os.path.isdir(pth+d):
            os.mkdir(pth+d)
      os.chdir(pth+d)

      for xi in xinits_list:
            model = build.Model(data=redata, kind='xradd', inits=xi, depends_on=depends_on)
            model.make_optimizer(tol=1.e-5, multiopt=True, maxfev=500)
            opt=model.opt
            opt.make_simulator()

            ydata, ywts = model.avg_y, model.avg_wts
            # STAGE 3 (Final Simplex)
            yhat, finfo, xiv = opt.gradient_descent(y=ydata, wts=ywts, inits=xi, is_flat=False)

            finfo_list.append(finfo)
            yhat_list.append(yhat)

Optimization terminated successfully.
         Current function value: 0.000016
         Iterations: 27
         Function evaluations: 66
Optimization terminated successfully.
         Current function value: 0.000275
         Iterations: 24
         Function evaluations: 57
Optimization terminated successfully.
         Current function value: 0.000015
         Iterations: 26
         Function evaluations: 66
Optimization terminated successfully.
         Current function value: 0.000116
         Iterations: 22
         Function evaluations: 55
Optimization terminated successfully.
         Current function value: 0.000015
         Iterations: 29
         Function evaluations: 76
Optimization terminated successfully.
         Current function value: 0.000018
         Iterations: 22
         Function evaluations: 61
Optimization terminated successfully.
         Current function value: 0.000016
         Iterations: 24
         Function evaluations: 60
Optimization terminated successful

In [7]:
dfx = pd.DataFrame([fi.T for fi in finfo_list])
dff = dfx[['cnvrg', 'logp', 'chi', 'rchi', 'AIC', 'BIC', 'a', 'v', 'tr_bsl', 'tr_pnl', 'ssv', 'xb', 'z']]
dff.to_csv('xradd_tr_bootinfo.csv', index=False)

yfits = np.vstack([np.vstack(yh.reshape(2,16)) for yh in yhat_list])
cond = ['bsl', 'pnl']*int(len(yfits)/2)
yhatdf = pd.DataFrame(yfits)
yhatdf.insert(0, 'Cond', cond)
yhatdf.to_csv('xradd_tr_bootfits.csv')